<h1> Capstone Project


This notebook is for the capstone assignment project.

In [12]:
import pandas as pd
import numpy as np

In [13]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


<h1>Week 3 - Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

<h2>~~~~~~~~~~~~~~ Part 1 ~~~~~~~~~~~~~~

In [2]:
import requests
!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.1       |   py36h9f0ad1d_0         163 KB  conda-forge
    soupsieve-2.0.1            |   py36h9f0ad1d_0          56 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         219 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     conda-forge/linux-64::beautifulsoup4-4.9.1-py36h9f0ad1d_0
  soupsieve          conda-forge/linux-64::soupsieve-2.0.1-py36h9f0ad1d_0



beautifulsoup4-4.9.1 | 163 KB    | ##################################### | 100% 
soupsieve-2.0.1      | 56 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction

In [23]:
!conda install -c conda-forge lxml --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libxslt-1.1.33             |       h7d1a2b0_0         426 KB
    lxml-3.8.0                 |           py36_0         3.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.2 MB

The following NEW packages will be INSTALLED:

  libxslt            pkgs/main/linux-64::libxslt-1.1.33-h7d1a2b0_0
  lxml               conda-forge/linux-64::lxml-3.8.0-py36_0



lxml-3.8.0           | 3.8 MB    | ##################################### | 100% 
libxslt-1.1.33       | 426 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


<b>Reading the wikipedia page and parsing the table and its columns

In [32]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup = BeautifulSoup(source)

In [33]:
pctable = soup.find('table',{'class':'wikitable sortable'})
print(pctable.tr.text)


Postal Code

Borough

Neighborhood



<b>Creating dataframe

In [39]:
column_names=['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns=column_names)

for tr_cell in pctable.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
        
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<b>Cleaning and grouping the dataframe

In [42]:
df = df[df.Borough!='Not assigned']
df = df[df.Borough!= 0]
df.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,df.shape[0]):
    if df.iloc[i][2] == 'Not assigned':
        df.iloc[i][2] = df.iloc[i][1]
        i = i+1
                          
df = df.dropna()
df = df[(df.Postalcode != 'Not assigned' ) & (df.Borough != 'Not assigned') & (df.Neighborhood != 'Not assigned')]       
df = df.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
print(df.shape)
df.head()

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h2>~~~~~~~~~~~~~~ Part 2 ~~~~~~~~~~~~~~

In [44]:
!conda install -c conda-forge geocoder --yes
import geocoder

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    click-7.1.2                |     pyh9f0ad1d_0          64 KB  conda-forge
    future-0.18.2              |   py36h9f0ad1d_1         714 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         837 KB

The following NEW packages will be INSTALLED:

  click              conda-forge/noarch::click-7.1.2-pyh9f0ad1d_0
  decorator          conda-forge/noarch::decorator-4.4.2-py_0
  future             conda-forge/linux-64::future-0.18.2-py36h9f0ad1d_1
  geocoder       

In [47]:
def getCoordinates(pCode):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(pCode))
        lat_lng_coords = g.latlng
    return lat_lng_coords


<b>Parsing the whole dataframe and calling the function from geocoder to get the coordinates for each row.

In [49]:
pCodes = df['Postalcode']    
coords = [getCoordinates(pCode) for pCode in pCodes.tolist()]
dfCoords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = dfCoords['Latitude']
df['Longitude'] = dfCoords['Longitude']
df.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.785779,-79.157368
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813
5,M1J,Scarborough,Scarborough Village,43.744203,-79.228725
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.726881,-79.265694
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.713340,-79.284942
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.723538,-79.228353
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696448,-79.265642


<h2>~~~~~~~~~~~~~~ Part 3 ~~~~~~~~~~~~~~

In [54]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Import finished.')

Solving environment: done

# All requested packages already installed.

Import finished.


<b> Getting only Toronto neighborhoods 

In [62]:
df2 = df[df['Borough'].str.contains('Toronto',regex=False)]
df2

,Postalcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.678148,-79.295349
41,M4K,East Toronto,"The Danforth West, Riverdale",43.683424,-79.354564
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668291,-79.315578
43,M4M,East Toronto,Studio District,43.648000,-79.339260
44,M4N,Central Toronto,Lawrence Park,43.729455,-79.386415
45,M4P,Central Toronto,Davisville North,43.713171,-79.388870
46,M4R,Central Toronto,North Toronto West,43.714139,-79.406456
47,M4S,Central Toronto,Davisville,43.703327,-79.385649
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.690328,-79.383522
49,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686378,-79.402372


<b> Placing the above neighborhoods on a map

In [63]:
torontoMap = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighborhood in zip(df2['Latitude'],df2['Longitude'],df2['Borough'],df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3196ad',
    fill_opacity=0.7,
    parse_html=False).add_to(torontoMap)
torontoMap

<b> Clustering with K-Means

In [64]:
k=5
clusterdf = df2.drop(['Postalcode','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(clusterdf)
kmeans.labels_
df2.insert(0, 'Cluster #', kmeans.labels_)
df2

,Cluster #,Postalcode,Borough,Neighborhood,Latitude,Longitude
37,3,M4E,East Toronto,The Beaches,43.678148,-79.295349
41,0,M4K,East Toronto,"The Danforth West, Riverdale",43.683424,-79.354564
42,3,M4L,East Toronto,"India Bazaar, The Beaches West",43.668291,-79.315578
43,0,M4M,East Toronto,Studio District,43.648000,-79.339260
44,2,M4N,Central Toronto,Lawrence Park,43.729455,-79.386415
45,2,M4P,Central Toronto,Davisville North,43.713171,-79.388870
46,2,M4R,Central Toronto,North Toronto West,43.714139,-79.406456
47,2,M4S,Central Toronto,Davisville,43.703327,-79.385649
48,2,M4T,Central Toronto,"Moore Park, Summerhill East",43.690328,-79.383522
49,2,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686378,-79.402372


<b> Mapping clusters

In [68]:
mapClusters = folium.Map(location=[43.651070,-79.347015],zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighborhood, clusterNumber in zip(df2['Latitude'], df2['Longitude'], df2['Neighborhood'], df2['Cluster #']):
    label = folium.Popup(' Cluster ' + str(clusterNumber), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[clusterNumber-1],
        fill=True,
        fill_color=rainbow[clusterNumber-1],
        fill_opacity=0.7).add_to(mapClusters)
       
mapClusters

<h1> The maps can be seen on the PDF Maps-Week3-Part3.pdf located on the root of the repository, since they probably can't be seen directly on github.